## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")



In [4]:
from langchain_groq import ChatGroq
model=ChatGroq(model="openai/gpt-oss-20b",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021633A5AE30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002164D992560>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))

LangChain uses structured message objects (HumanMessage, AIMessage, SystemMessage) instead of raw text.

This enables multi-turn context directly.

LangChain defines different message classes to represent different roles or sources of a message.

These are:
* HumanMessage 👤 — what the user says
* AIMessage 🤖 — what the AI responds
* SystemMessage ⚙️ — instructions or context for the model

In [43]:
from langchain_core.messages import HumanMessage, AIMessage
model.invoke([HumanMessage(content="Hi , My name is Jhon and I am an AI Engineer")])

AIMessage(content='Hello Jhon! 👋 It’s great to meet an AI Engineer. How can I assist you today? Whether you’re working on a project, need insights on the latest research, or just want to brainstorm ideas, I’m here to help.', additional_kwargs={'reasoning_content': 'User says "Hi , My name is Jhon and I am an AI Engineer". They introduced themselves. We should respond. The user is Jhon. We can ask what they\'d like to discuss or help with. Also note that the user might want a conversation. We\'ll greet them and ask how we can help.'}, response_metadata={'token_usage': {'completion_tokens': 123, 'prompt_tokens': 84, 'total_tokens': 207, 'completion_time': 0.123429981, 'prompt_time': 0.003983243, 'queue_time': 0.00310321, 'total_time': 0.127413224}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_80501ff3a1', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--b85ddd70-8e7e-4fe2-b595-47ae5c75c3c4-0', usage_metadata={

In [ ]:

model.invoke(
    [
        HumanMessage(content="Hi , My name is Jhon and I am an AI Engineer"),
        AIMessage(content="Hello Jhon! It's nice to meet you. \n\nAs an AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content='I’m not sure what your name is or what you do—could you let me know a bit more? Once I have that, I’ll be happy to help or chat about it!', additional_kwargs={'reasoning_content': 'The user says: "Hey What\'s my name and what do I do?" There\'s no context. The user presumably wants to know their name and what they do. But we don\'t have that info. We should ask for clarification. We can respond: I don\'t know your name or what you do, but I\'d be happy to help if you tell me. Probably the best answer: I\'m not aware of your name or what you do; please provide more context.'}, response_metadata={'token_usage': {'completion_tokens': 139, 'prompt_tokens': 81, 'total_tokens': 220, 'completion_time': 0.137904112, 'prompt_time': 0.005100446, 'queue_time': 0.033860283, 'total_time': 0.143004558}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

* You create a store (dictionary) that maps session IDs to chat histories.

* RunnableWithMessageHistory automatically loads/saves conversation messages between turns.

* Now, your model “remembers” previous messages per session.

In [9]:
!pip install langchain_community


  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
Using cached langchain_community-0.4.1-py3-none-any.whl (2.5 MB)


In [ ]:
# in-memory message store.
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

#wraps any chain or model
from langchain_core.runnables.history import RunnableWithMessageHistory

#session store
store={}


#Checks whether the session (like "chat1") already has a history.
#If not, it creates a new empty one.
#Returns the history object for that session.
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]


#wraps your LLM model (model) with memory behavior
with_message_history=RunnableWithMessageHistory(model,get_session_history)

Every time you call invoke() on with_message_history, LangChain automatically:
* Looks up the session using get_session_history
* Loads all previous messages for that session
* Adds the new user input (HumanMessage)
* Sends everything to the model
* Saves the new model response (AIMessage) back into the same history

Now our model “remembers” earlier turns automatically.

In [47]:
# which session to use when calling invoke()

config={"configurable":{"session_id":"chat1"}}

#The key "configurable" is required by LangChain’s configuration system.
#The "session_id" identifies which conversation history to use.
#If you pass "chat2" instead, it will create a new independent chat memory.

In [48]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Jhon and I am an AI Engineer")],
    config=config
)

WITH MESSAGEhistory(config(sessionid))->model+getsession

What happens here:
* LangChain sees session_id = "chat1".
* It calls get_session_history("chat1") → new history is created.
* The input message (HumanMessage) is added to the history.
* The model generates a reply (AIMessage).
* The AIMessage is stored back in the same history.

In [49]:
response.content

'Hello, Jhon! 👋 It’s great to meet another AI enthusiast. How can I assist you today? Whether you’re working on a new project, troubleshooting code, or just curious about the latest in AI, I’m here to help!'

In [50]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='You mentioned your name is **Jhon**.', additional_kwargs={'reasoning_content': 'User says "My name is Jhon". So answer is Jhon.'}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 148, 'total_tokens': 183, 'completion_time': 0.0351388, 'prompt_time': 0.009955989, 'queue_time': 0.002859986, 'total_time': 0.045094789}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--0d4e02e2-e87d-401f-90bb-340f1ed9485f-0', usage_metadata={'input_tokens': 148, 'output_tokens': 35, 'total_tokens': 183})

What happens now:
* LangChain again uses the same session_id = "chat1".
* It loads previous messages from history (the “Hi, my name is …” exchange).
* It appends the new HumanMessage("What's my name?").
* Sends the full conversation to the model.
* The model can recall  name from the earlier part of the chat.

In [ ]:
## change the config-->session id
config11={"configurable":{"session_id":"chat22"}} #config->{"configurable":{"session_id":"________"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config11
)
response.content

'I don’t actually have that information. Could you let me know what you’d like me to call you?'

In [56]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Jack")],
    config=config11
)
response.content

'Nice to meet you, Jack! How can I help you today?'

In [58]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John.'

---

| Component                                        | What it does                        | Key Role                        |
| ------------------------------------------------ | ----------------------------------- | ------------------------------- |
| `ChatMessageHistory`                             | Stores messages in memory           | Acts like chat memory           |
| `BaseChatMessageHistory`                         | Interface type                      | Ensures consistent structure    |
| `store = {}`                                     | In-memory session map               | Holds all chat sessions         |
| `get_session_history()`                          | Retrieves or creates session memory | Session-level memory management |
| `RunnableWithMessageHistory`                     | Wraps model to add memory logic     | Automatically remembers chats   |
| `config = {"configurable": {"session_id": ...}}` | Identifies chat session             | Links calls to the same memory  |
| `invoke()`                                       | Runs the model with message history | Main entry point                |


### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

---

You define a system message for global behavior.

MessagesPlaceholder acts as a slot where conversation history (human + AI messages) will be inserted dynamically.

The | operator chains components (prompt → model).

In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [ ]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Jhon")]})

AIMessage(content='Hello Jhon! 👋 How can I help you today?', additional_kwargs={'reasoning_content': 'User says "Hi My name is Jhon". Likely greeting. Need to respond. Possibly ask how to help.'}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 98, 'total_tokens': 145, 'completion_time': 0.046152953, 'prompt_time': 0.004692686, 'queue_time': 0.003334207, 'total_time': 0.050845639}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--35b24273-4aad-4f74-afa7-e79592c70398-0', usage_metadata={'input_tokens': 98, 'output_tokens': 47, 'total_tokens': 145})

combine this with your RunnableWithMessageHistory:

In [20]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [21]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Jhon")],
    config=config
)

response

AIMessage(content='Hello Jhon! 👋 How can I help you today?', additional_kwargs={'reasoning_content': 'We need to respond. The user says "Hi My name is Jhon". Likely a greeting. We can reply. The instruction says "You are a helpful assistant. Answer all the question to the best of your ability." There\'s no question. Just greet. We can respond politely. Maybe ask how can help.'}, response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 98, 'total_tokens': 185, 'completion_time': 0.086407487, 'prompt_time': 0.005399963, 'queue_time': 0.002957473, 'total_time': 0.09180745}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_80501ff3a1', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--91069f6c-eb1a-40f1-8d21-33c496399a49-0', usage_metadata={'input_tokens': 98, 'output_tokens': 87, 'total_tokens': 185})

In [22]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jhon.'

In [23]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [24]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Jhon")],"language":"Malayalam"})
response.content

'ഹായ് ജോൺ! 👋 എനിക്ക് നിങ്ങളെ എങ്ങനെ സഹായിക്കാം?'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [84]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [85]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Jake")],"language":"Malayalam"},
    config=config
)
repsonse.content

'ഹായ് ജേക്ക്! എങ്ങനെയുണ്ട്? ഇന്ന് എനിക്ക് നിങ്ങളെ എങ്ങനെ സഹായിക്കാം?'

In [88]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Arabic"},
    config=config,
)

In [89]:
response.content

'عذراً، لا أعرف اسمك. إذا رغبت في إخباري، سأكون سعيداً بمعرفة اسمك!'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [122]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=40,
    strategy="last", #first last middle
    token_counter=model, # how tokens are counted
    include_system=True, #whether the system messageshould always be kept.
    allow_partial=False,#whether a single long message can be partially truncated
    start_on="human" #where to start counting tokens from
)

#trim_messages() returns a Runnable (a LangChain “mini-pipeline” object)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [123]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

In [124]:


response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

'I’m not sure—could you tell me a bit about your favorite flavors? For example, do you prefer classic vanilla, chocolate, or something more adventurous like salted caramel or pistachio? Any ingredients you love or dislike? That’ll help me guess!'

In [125]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You haven’t asked a math problem yet—just chatting! If you have one in mind, feel free to share it.'

In [121]:
print(response.content)

You asked: **“What is 2 + 2?”**


In [109]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [111]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'I don’t know—could you let me know what you’d like to be called?'

In [112]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'I’m not sure which math problem you’re referring to. Could you let me know the details or the question you had in mind? That way I can help you more accurately.'

| Concept          | Tool / Class                 | Purpose                           |
| ---------------- | ---------------------------- | --------------------------------- |
| Model            | `ChatGroq`                   | Connects to Groq API              |
| Stateful memory  | `RunnableWithMessageHistory` | Keeps chat context                |
| Prompts          | `ChatPromptTemplate`         | Defines structure + system role   |
| History trimming | `trim_messages()`            | Prevents context overflow         |
| Multi-language   | `{language}` in prompt       | Makes responses language-specific |
